<a href="https://www.kaggle.com/code/denisndemwa/denis-ndemwa-netflix-data-wrangling?scriptVersionId=249977727" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/netflix-shows/netflix_titles.csv


Data Science Project: Data Wrangling
This project showcases my walkthrough for data wrangling using python on netflix data.

The steps that I will work through are:
1. Discovery to understand the data, its existing format and quality.
2. Structuring to understand and standardize the formats.
3. Cleaning
   * Remove Duplicate
   * Remove Irrelevant Information
   * Handle missing Values
   * Handle Outliers
4. Enriching
5. Validating
6. Publishing

# Step 1:Discovery
This is the initial stage where I will explore and understand the data. 
Key activities include:
1. Reviewing data sources and formats.
2. Understanding the schema, structure, and relationships.
3. Assessing data quality by checking for completeness, accuracy, consistency, and reliability.
4. Identifying potential data issues such as missing values, duplicates, or inconsistencies.

In [2]:
#import the Data to a Pandas DataFrame
df = pd.read_csv('/kaggle/input/netflix-shows/netflix_titles.csv')

#Have a quick overview of the data
df.info()

df.describe()

# Number of rows and columns
print("Shape of the dataset (R x C):", df.shape)
# List of all column names
print("Columns in the dataset:\n", df.columns.tolist())
# Data types of each column
print("Data types:\n", df.dtypes)
# Group and Count of missing (null) values in each column
print("Missing values per column:\n", df.isnull().sum())
# Group and Count of duplicate rows
print("Number of duplicate rows:", df.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB
Shape of the dataset (R x C): (8807, 12)
Columns in the dataset:
 ['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description']
Data types:
 show_id         object
type      

# Step 2: Structuring
In this step, the goal is to organize and format the data consistently for easier processing:
1. Standardizing column names, data types, and formats.
2. Flattening nested structures.
3. Pivoting/unpivoting tables, combining multiple sources into a uniform schema.
4. Ensuring the data is logically structured to suit downstream analysis or modelling.

In [3]:
# Convert 'date_added' to datetime
df['date_added'] = pd.to_datetime(df['date_added'],format='mixed')

# Separate 'duration' into numeric value and unit (e.g., '90 min' → 90, 'min')
df[['duration_value', 'duration_unit']] = df['duration'].str.extract(r'(\d+)\s*(\w+)')

# Convert duration_value to numeric
df['duration_value'] = pd.to_numeric(df['duration_value'])

# View Resulting columns
print(df[['duration_value', 'duration_unit']])

      duration_value duration_unit
0               90.0           min
1                2.0       Seasons
2                1.0        Season
3                1.0        Season
4                2.0       Seasons
...              ...           ...
8802           158.0           min
8803             2.0       Seasons
8804            88.0           min
8805            88.0           min
8806           111.0           min

[8807 rows x 2 columns]


# Step 3: Cleaning
This critical phase involves removing or correcting inaccurate or problematic data:
1. Check and remove Duplicates: Eliminate repeated records that can skew analysis.
2. Remove Irrelevant Information: Drop columns or rows that don't contribute to the analysis.
3. Handle Missing Values:
    * Remove rows/columns with excessive missingness.
    * Impute missing data using mean, median, mode, or predictive techniques.
4. Handle Outliers:
      * Detect extreme values using statistical methods.
      * Remove or cap/floor outliers based on the business context.

In [4]:
# Check for duplicate rows
print("Duplicate rows before:", df.duplicated().sum())

# Drop duplicate rows
df = df.drop_duplicates()

# Drop description column
df = df.drop(columns=['description'])

# Impute missing 'director' values using frequent director-cast combinations
df['dir_cast'] = df['director'] + '---' + df['cast']
counts = df['dir_cast'].value_counts()
filtered_counts = counts[counts >= 3]
lst_dir_cast = list(filtered_counts.index)

dict_direcast = {}
for i in lst_dir_cast:
    director, cast = i.split('---')
    dict_direcast[director] = cast

for director, cast in dict_direcast.items():
    df.loc[(df['director'].isna()) & (df['cast'] == cast), 'director'] = director

# Assign 'Not Given' to all remaining missing directors
df.loc[df['director'].isna(), 'director'] = 'Not Given'

# Use director to fill missing countries
directors = df['director']
countries = df['country']
pairs = zip(directors, countries)
dir_cntry = dict(pairs)

for director, country in dir_cntry.items():
    df.loc[(df['country'].isna()) & (df['director'] == director), 'country'] = country

# Assign 'Not Given' to remaining missing countries
df.loc[df['country'].isna(), 'country'] = 'Not Given'

# Assign 'Not Given' to remaining missing cast entries
df.loc[df['cast'].isna(), 'cast'] = 'Not Given'

# Drop rows with missing critical values
df.dropna(subset=['date_added', 'rating', 'duration'], inplace=True)

# Fixing release year vs date_added inconsistencies
import datetime as dt

# Ensure 'date_added' is in datetime format
df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')

# Check for inconsistencies
print("Inconsistent date_added vs release_year count:", sum(df['date_added'].dt.year < df['release_year']))

# Show rows with inconsistencies
print(df.loc[df['date_added'].dt.year < df['release_year'], ['date_added', 'release_year']])

# Sample rows for manual inspection
print(df.iloc[[1551, 1696, 2920, 3168]])

# Confirm no more release_year inconsistencies
print("Remaining inconsistencies:", sum(df['date_added'].dt.year < df['release_year']))

Duplicate rows before: 0
Inconsistent date_added vs release_year count: 14
     date_added  release_year
1551 2020-12-14          2021
1696 2020-11-15          2021
2920 2020-02-13          2021
3168 2019-12-06          2020
3287 2019-11-13          2020
3369 2019-10-25          2020
3433 2019-10-11          2020
4844 2018-05-30          2019
4845 2018-05-29          2019
5394 2017-07-01          2018
5658 2016-12-23          2018
5677 2016-12-13          2017
7063 2018-10-26          2019
7112 2013-03-31          2016
     show_id     type          title   director  \
1551   s1552  TV Show          Hilda  Not Given   
1696   s1697  TV Show   Polly Pocket  Not Given   
2920   s2921  TV Show  Love Is Blind  Not Given   
3168   s3169  TV Show   Fuller House  Not Given   

                                                   cast  \
1551  Bella Ramsey, Ameerah Falzon-Ojo, Oliver Nelso...   
1696  Emily Tennant, Shannon Chan-Kent, Kazumi Evans...   
2920                        Nick Lachey, V

# Step 4: Enriching
In this stage, I will enhance the dataset by adding meaningful information:
1. Merge with external datasets 
2. Derive new features or calculated fields 
3. Categorize or bin continuous variables


In [5]:
import pandas as pd              # For data manipulation and analysis
import numpy as np               # For numerical computations
import datetime as dt            # For handling date and time
import logging                   # For logging messages and debugging
import seaborn as sns            # For statistical data visualization
import matplotlib.pyplot as plt  # For general plotting

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

def remove_duplicates(df):
    logging.info(f"Duplicate rows before: {df.duplicated().sum()}")
    return df.drop_duplicates()

def drop_unused_columns(df):
    if 'description' in df.columns:
        df.drop(columns=['description'], inplace=True)
    return df

def impute_director(df):
    df['dir_cast'] = df['director'].astype(str) + '---' + df['cast'].astype(str)
    freq_pairs = df['dir_cast'].value_counts()
    reliable_pairs = freq_pairs[freq_pairs >= 3].index

    for pair in reliable_pairs:
        director, cast = pair.split('---')
        df.loc[(df['director'].isna()) & (df['cast'] == cast), 'director'] = director
    
    df['director'].fillna('Not Given', inplace=True)
    df.drop(columns=['dir_cast'], inplace=True)
    return df

def impute_country(df):
    dir_cntry_map = df.dropna(subset=['director', 'country']).drop_duplicates(subset=['director']) \
                      .set_index('director')['country'].to_dict()
    df['country'] = df.apply(
        lambda x: dir_cntry_map.get(x['director'], 'Not Given') if pd.isna(x['country']) else x['country'], axis=1
    )
    return df

def fill_missing_simple(df):
    df['cast'].fillna('Not Given', inplace=True)
    return df

def drop_rows_with_critical_nulls(df):
    return df.dropna(subset=['date_added', 'rating', 'duration'])

def fix_and_validate_dates(df):
    df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')
    inconsistencies = df[df['date_added'].dt.year < df['release_year']]
    logging.info(f"Inconsistent 'date_added' < 'release_year' rows: {len(inconsistencies)}")
    logging.debug(inconsistencies[['date_added', 'release_year']])
    return df

def plot_missing_data(df):
    plt.figure(figsize=(10, 6))
    sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
    plt.title('Missing Data Heatmap')
    plt.show()

def main(df):
    df = remove_duplicates(df)
    df = drop_unused_columns(df)
    df = impute_director(df)
    df = impute_country(df)
    df = fill_missing_simple(df)
    df = drop_rows_with_critical_nulls(df)
    df = fix_and_validate_dates(df)
    plot_missing_data(df)
    logging.info("Data cleaning complete.")
    return df

# Example usage
# df = pd.read_csv("your_dataset.csv")
# df_cleaned = main(df)

# Step 5: Validating
After cleaning and enriching, data is verified for consistency, accuracy, and integrity:
1. Remove temporary or wrangling helper columns.
2. Ensure correct data types and extracting duration value (numeric) and unit if needed.
3. Business logic: Identify records before 1997 (Netflix launch year).
4. Check for missing values in key fields.
5. Sample a few rows to visually inspect the data
6. Reset the index
7. Confirm data types

In [6]:
# 1. Remove temporary or wrangling helper columns
columns_to_drop = ['dir_cast', 'duration_value', 'duration_unit']
df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

# 2. Ensure correct data types
df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')

# Extract duration value (numeric) and unit if needed
if 'duration' in df.columns:
    df['duration_value'] = df['duration'].str.extract(r'(\d+)').astype(float)
    df['duration_unit'] = df['duration'].str.extract(r'([a-zA-Z]+)').astype(str)

# 3. Business logic: Identify records before 1997 (Netflix launch year)
pre_1997 = df[df['release_year'] < 1997]
print(f"\n🔍 Records before 1997: {len(pre_1997)}")
if not pre_1997.empty:
    print(pre_1997[['title', 'release_year']].head())

# 4. Check for missing values in key fields
key_fields = ['director', 'cast', 'country', 'date_added', 'rating', 'duration']
missing_info = df[key_fields].isna().sum()
print("\n🧹 Missing values in key fields:")
print(missing_info[missing_info > 0])

# 5. Sample a few rows to visually inspect the data
print("\n🔎 Sample records:")
print(df.sample(5))

# 6. Reset the index
df = df.reset_index(drop=True)

# 7. Confirm data types
print("\n📊 Column data types:")
print(df.dtypes)



🔍 Records before 1997: 412
              title  release_year
7           Sankofa          1993
22  Avvai Shanmughi          1996
41             Jaws          1975
42           Jaws 2          1978
43           Jaws 3          1983

🧹 Missing values in key fields:
Series([], dtype: int64)

🔎 Sample records:
     show_id     type              title           director  \
456     s457  TV Show   Her Private Life          Not Given   
4216   s4217  TV Show            Déjà Vu          Not Given   
8461   s8462    Movie  The Polar Express    Robert Zemeckis   
8707   s8708    Movie    We, the Marines  Greg MacGillivray   
7464   s7465    Movie       Miss Hokusai       Keiichi Hara   

                                                   cast        country  \
456   Park Min-young, Kim Jae-uk, Ahn Bo-hyun, Jung ...    South Korea   
4216  Mandy Wei, Yao Yuan Hao, Wang Si Ping, Yang Zh...      Not Given   
8461  Tom Hanks, Leslie Zemeckis, Eddie Deezen, Nona...  United States   
8707            

# Step 6: Publishing
Finally, the wrangled data is exported or made accessible for analysis or usage:
1. Save in desired formats which is CSV in this case.
2. Store in data warehouses, lakes, or BI tools.
3. Ensure metadata and documentation are provided for users.
4. Set up pipelines for automation if the wrangling process is repeated regularly

In [7]:
# Save as CSV 
df.to_csv('/kaggle/working/cleaned_netflix.csv', index=False)